In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [2]:
base = pd.read_csv('test.csv')
base2 = pd.read_csv('test2.csv')

In [3]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
x2 = base2.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe
y2 = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [4]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape, y2.shape

((60,), (60,))

In [5]:
y = y.values.reshape(-1,1)
y2 = y2.values.reshape(-1,1)
y.shape, y2.shape

((60, 1), (60, 1))

In [6]:
y = onehot.fit_transform(y).toarray()
y2 = onehot.fit_transform(y2).toarray()
y, y2

(array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0

In [7]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.3)

In [8]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((60, 100), (18, 100), (60, 3), (18, 3))

In [9]:
neuronios_entrada = x.shape[1]
neuronios_entrada

100

In [10]:
neuronios_oculta = int((neuronios_entrada + y.shape[1])/2)
neuronios_oculta

51

In [11]:
neuronios_saida = y.shape[1]
neuronios_saida

3

In [12]:
w = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}
w

{'oculta': <tf.Variable 'Variable:0' shape=(100, 51) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_1:0' shape=(51, 3) dtype=float32_ref>}

In [13]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta': <tf.Variable 'Variable_2:0' shape=(51,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_3:0' shape=(3,) dtype=float32_ref>}

In [14]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [15]:
def mlp(x, w, bias):
    camada_oculta = tf.add(tf.matmul(x, w['oculta']), b['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']), b['saida'])
    return camada_saida

In [16]:
modelo = mlp(xph, w, b)

In [17]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(erro)

In [18]:
batch_size = 4

In [19]:
batch_total = int(len(x_treinamento) / batch_size)
batch_total

10

In [20]:
x_batches = np.array_split(x_treinamento, batch_total)
x_batches

[    Amostra 1  Amostra 2  Amostra 3  Amostra 4  Amostra 5  Amostra 6  \
 53        4.9      4.900      4.900      4.900      4.900      4.900   
 3         0.0      0.507      1.013      1.514      2.009      2.496   
 28        0.0      0.900      1.800      2.700      3.600      4.500   
 34        3.0      3.500      4.000      4.500      5.000      5.500   
 12        0.0      0.308      0.614      0.917      1.215      1.506   
 
     Amostra 7  Amostra 8  Amostra 9  Amostra 10  ...  Amostra 91  Amostra 92  \
 53      4.900      4.900     -4.900      -4.900  ...       4.900       4.900   
 3       2.973      3.438      3.890       4.325  ...      -4.325      -3.890   
 28      5.400      6.300      7.200       8.100  ...       9.000       9.900   
 34      6.000      6.500      7.000       7.500  ...       8.000       8.500   
 12      1.790      2.063      2.326       2.577  ...       1.189       1.481   
 
     Amostra 93  Amostra 94  Amostra 95  Amostra 96  Amostra 97  Amostra

In [21]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(3000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])

Época: 1 erro: 500.797702217102
Época: 501 erro: 0.006558258078712065
Época: 1001 erro: 0.00016377875249400374
Época: 1501 erro: 1.5134807918393278e-05
Época: 2001 erro: 1.6033443264618083e-06
Época: 2501 erro: 1.5974027700593751e-07


In [22]:
w_final, b_final

({'oculta': array([[ 1.0728003 , -1.4121037 , -0.61145055, ...,  0.56054133,
          -0.45589295,  1.2453874 ],
         [-0.3448142 ,  0.70872086, -1.8321002 , ...,  0.02802613,
          -0.91244185, -0.31701994],
         [ 1.376421  ,  1.259135  ,  0.7453335 , ..., -0.41263005,
           0.35199562,  1.566064  ],
         ...,
         [-1.2912402 ,  0.08424082, -0.05917333, ...,  1.8374581 ,
           0.6558247 , -0.18700925],
         [-0.01073581, -1.5602597 , -0.9031249 , ...,  0.68813425,
          -0.4821496 ,  1.392388  ],
         [-1.237777  , -1.4877526 ,  0.6055094 , ...,  0.420113  ,
          -0.61231494,  0.23796734]], dtype=float32),
  'saida': array([[ 1.0151395 ,  0.56485844, -0.9612332 ],
         [ 0.35467473, -0.71466196,  1.1050595 ],
         [-0.48696107, -0.5437292 , -1.6790813 ],
         [-0.5300723 , -0.462734  ,  1.2291472 ],
         [-0.5022376 ,  0.86861736,  0.6046727 ],
         [-0.6397207 , -1.0019268 ,  0.03950358],
         [-2.5378377 , -0.

In [23]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))

In [24]:
r1

array([[-254.87193 ,    7.385465,  244.56656 ],
       [ 332.17252 ,   58.894054,  155.46364 ],
       [-486.5937  , -510.73184 ,   65.95942 ],
       [  60.910828,  388.85947 ,  272.93393 ],
       [  23.429855,  126.12574 ,   88.476425],
       [ 167.11926 ,   28.249063,   73.244835],
       [  29.679184,  170.04097 ,  119.3123  ],
       [  28.620682,  -40.567146, -159.33755 ],
       [  15.618157,   71.23159 ,   49.93176 ],
       [-183.12712 , -195.28099 ,   20.544308],
       [-607.9802  , -636.91205 ,   84.125336],
       [  20.3051  ,  104.16804 ,   73.05866 ],
       [ 299.1549  ,   52.756565,  139.0453  ],
       [  14.055787,   60.252743,   42.222813],
       [ 209.505   , -416.74252 ,  255.57468 ],
       [ 334.48227 ,   59.311214,  156.62901 ],
       [-859.91614 , -516.19794 ,  712.0205  ],
       [  26.554611,  148.08344 ,  103.89432 ]], dtype=float32)

In [25]:
r2

array([[0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 4.4576945e-17],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 9.3072056e-23],
       [1.0000000e+00, 8.9558752e-31, 0.0000000e+00],
       [7.0370810e-25, 1.0000000e+00, 5.6182459e-10],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [3.7913060e-37, 1.0000000e+00, 3.0858166e-14],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [8.6480190e-21, 1.0000000e+00, 1.4780899e-08],
       [9.8219071e-21, 0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 6.4403597e-20]], dtype=float32)

In [26]:
r3

array([2, 0, 2, 1, 1, 0, 1, 0, 1, 2, 2, 1, 0, 1, 2, 0, 2, 1], dtype=int64)

In [27]:
y_teste

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [28]:
y_teste2 = np.argmax(y_teste, 1)
y_teste2

array([2, 0, 2, 1, 1, 0, 1, 2, 1, 2, 2, 1, 0, 1, 2, 0, 0, 1], dtype=int64)

In [29]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto

0.8888888888888888